In [ ]:
from google.colab import files
uploaded=files.upload()

Saving T1.csv to T1.csv


In [ ]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """


In [ ]:
window_size = 48
batch_size_exp = 1
epoch_exp = 7
neurons_exp = 50
predict_values_exp = 6000
lag_exp=72

In [ ]:
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [ ]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

In [ ]:
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [ ]:
def scale(data_norm):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(data_norm)
    # transform train
    data_norm = data_norm.reshape(data_norm.shape[0], data_norm.shape[1])
    data_scaled = scaler.transform(data_norm)
    return scaler, data_scaled

In [ ]:
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [ ]:
from keras.layers import Activation, Dense, BatchNormalization, TimeDistributed
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons_exp, dropout = 0.1 ,batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(BatchNormalization())
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dense(50))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
    return model

In [ ]:
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    #print(X)
    yhat = model.predict(X, batch_size=1)
    return yhat[0,0]

In [ ]:
import pandas as pd
series = pd.read_csv('T1.csv',index_col="Date/Time")
series.head()

,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
Date/Time,,,,
01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


In [ ]:
#del series['Air temperature | (\'C)']
del series['LV ActivePower (kW)']
del series['Wind Direction (°)']
del series['Theoretical_Power_Curve (KWh)']
series.head()

,Wind Speed (m/s)
Date/Time,
01 01 2018 00:00,5.311336
01 01 2018 00:10,5.672167
01 01 2018 00:20,5.216037
01 01 2018 00:30,5.659674
01 01 2018 00:40,5.577941


In [ ]:
for i in range(0,10):
  series = series[:-1]
series.tail()

,Wind Speed (m/s)
Date/Time,
31 12 2018 21:30,8.472826
31 12 2018 21:40,8.433422
31 12 2018 21:50,9.976660
31 12 2018 22:00,10.416830
31 12 2018 22:10,10.713670


In [ ]:
series.shape

(50520, 1)

In [ ]:
raw_values = series.values
diff_values = difference(raw_values, 1)

In [ ]:
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, lag_exp)
supervised_values = supervised.values

In [ ]:
# split data into train and test-sets
scaler,supervised_values = scale(supervised_values)
train_scaled, test_scaled = supervised_values[0:-predict_values_exp], supervised_values[-predict_values_exp:]
#print(test_scaled)

In [ ]:
lstm_model = fit_lstm(train_scaled, batch_size_exp, epoch_exp, neurons_exp)

44519/44519 [==============================] - 137s 3ms/step - loss: 0.0018


In [ ]:
import pandas as pd
series = pd.read_csv('T1.csv',index_col="Date/Time")

In [ ]:
# walk-forward validation on the test data
predictions = list()
expectations = list()
for i in range(len(test_scaled)-window_size):
    window_prediction_frame = test_scaled
    test_pred = list()
    for j in range(window_size):
        X, y = window_prediction_frame[i, 0:-1], window_prediction_frame[i, -1]
        yhat = forecast_lstm(lstm_model, 1, X)#batch_size_exp to 1
        '''# Start Debug prints
        print("X: %", X)
        print("yhat: %", yhat)
        # End Debug prints'''
        # Replacing value in test scaled with the predicted value.
        test_pred = [yhat] + test_pred 
        if len(test_pred) > lag_exp+1:
            test_pred = test_pred[:-1]
        if j+1<len(window_prediction_frame):
            if j+1 > lag_exp+1:
                window_prediction_frame[j+1] = test_pred
            else:
                window_prediction_frame[j+1] = np.concatenate((test_pred, window_prediction_frame[j+1, j+1:]),axis=0)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 27ms/step


In [ ]:
 # invert scaling
    yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    # store forecast
    expected = raw_values[len(train_scaled) + i + 1 + window_size]
    predictions.append(yhat)
    expectations.append(expected)
    #print('Hour=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

In [ ]:
expectations = np.array(expectations)
predictions = np.array(predictions)
print("Mean Absolute Percent Error: ", (np.mean(np.abs((expectations - predictions) / expectations.mean()))*100))

Mean Absolute Percent Error:  49.85802971120778
